In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
  inflating: data/ratings.csv        
  inflating: data/movies.csv         


# Building a Movie Recommendation System in PySpark - Lab Code-along
![images of vhs tapes on shelf](img/movies.jpg)

## Introduction

In this last lab, we will implement a a movie recommendation system using Alternating Least Squares (ALS) in Spark programming environment.<br> Spark's machine learning libraray `ml` comes packaged with a very efficient imeplementation of ALS algorithm. 

The lab will require you to put into pratice your spark programming skills for creating and manipulating pyspark DataFrames. We will go through a step-by-step process into developing a movie recommendation system using ALS and pyspark using the MovieLens Dataset.

Note: You are advised to refer to [PySpark Documentation](http://spark.apache.org/docs/2.2.0/api/python/index.html) heavily for completing this lab as it will introduce a few new methods. 

## Objectives

You will be able to:

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* Run a brief hyper-parameter selection activity through a scalable grid search
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Building a Recommendation System

We have seen how recommender/Recommendation Systems have played an  integral parts in the success of Amazon (Books, Items), Pandora/Spotify (Music), Google (News, Search), YouTube (Videos) etc.  For Amazon these systems bring more than 30% of their total revenues. For Netflix service, 75% of movies that people watch are based on some sort of recommendation.

> The goal of Recommendation Systems is to find what is likely to be of interest to the user. This enables organizations to offer a high level of personalization and customer tailored services.

### We sort of get the concept

For online video content services like Netflix and Hulu, the need to build robust movie recommendation systems is extremely important. An example of recommendation system is such as this:

1.    User A watches Game of Thrones and Breaking Bad.
2.    User B performs a search query for Game of Thrones.
3.    The system suggests Breaking Bad to user B from data collected about user A.


This lab will guide you through a step-by-step process into developing such a movie recommendation system. We will use the MovieLens dataset to build a movie recommendation system using the collaborative filtering technique with Spark's Alternating Least Saqures implementation. After building that recommendation system, we will go through the process of adding a new user to the dataset with some new ratings and obtaining new recommendations for that user.

## Will Nightengale like Toy Story?

Collaborative filtering and matrix decomposition allows us to use the history of others ratings, along with the entire community of ratings, to answer that question.

![image1](img/collab.png)


## Person vs vegetable

It's important to realize that there are two sides to recommendation

![image2](img/item_user_based.png)

## Code for model

If we wanted, we could jump to the code right now to make this happen.

But would we understand it?
```python
from pyspark.ml.recommendation import ALS

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

als_model = als.fit(movie_ratings)
```

## Documentation Station

Let's explore the [documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#module-pyspark.ml.recommendation) together to maybe get a better idea of what is happening. 

- which parameters make sense?
- which are completely foreign?

## Rank

What's all this rank of the factorization business?<br>
[the source code documentation](https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/mllib/recommendation/ALS.scala) describes that variable as the "Rank of the feature matrices"

## Assumptions

Matrix decomposition is built on the theory that every individual (user, movie) score is actually the **dot product** of two separate vectors:
- user characteristics 
- movie characteristics

Wait, do you mean like gender, whether the movie is sci-fi or action? do we have that data?

![beyonce-gif](img/beyonce.gif)

## The hidden matricies 
![image4](img/matrix_decomp.png)

## Embeddings

Embeddings are low dimensional hidden factors for items and users.

For e.g. say we have 5 dimensional (i.e., **rank** = 5) embeddings for both items and users (5 chosen randomly, this could be any number - as we saw with PCA and dim. reduction).

For user-X & movie-A, we can say those 5 numbers might represent 5 different characteristics about the movie e.g.:

- How much movie-A is political
- How recent is the movie
- How much special effects are in movie A
- How dialogue driven is the movie
- How linear is the narrative in the movie

In a similar way, 5 numbers in the user embedding matrix might represent:

- How much does user-X like sci-fi movies
- How much does user-X like recent movies … and so on.

But we have *no actual idea* what those factors actually represent.

### If we knew the feature embeddings in advance, it would look something like this:

In [57]:
import numpy as np

# the original matrix of rankings
R = np.array([[2, np.nan, np.nan, 1, 4],
              [5, 1, 2, np.nan, 2],
              [3, np.nan, np.nan, 3, np.nan],
              [1, np.nan, 4, 2, 1]])

# users X factors
P = np.array([[-0.63274434,  1.33686735, -1.55128517],
              [-2.23813661,  0.5123861,  0.14087293],
              [-1.0289794,  1.62052691,  0.21027516],
              [-0.06422255,  1.62892864,  0.33350709]])

# factors X items
Q = np.array([[-2.09507374,  0.52351075,  0.01826269],
              [-0.45078775, -0.07334991,  0.18731052],
              [-0.34161766,  2.46215058, -0.18942263],
              [-1.0925736,  1.04664756,  0.69963111],
              [-0.78152923,  0.89189076, -1.47144019]])

What about that `np.nan` in the third row, last column? How will that item be reviewed by that user?

In [59]:
P[2].dot(Q.T[:,4])

1.9401031341455333

## Wait, I saw a transpose in there - what's the actual formula?

Terms:<br>
$R$ is the full user-item rating matrix

$P$ is a matrix that contains the users and the k factors represented as (user,factor)

$Q^T$ is a matrix that contains the items and the k factors represented as

$r̂_{u,i}$ represents our prediction for the true rating $r_{ui}$ In order to get an individual rating, you must take the dot product of a row of P and a column of Q

for the entire matrix:
$$ R = PQ^T $$ 

or for individual ratings

$$r̂_{u,i}=q_i^⊤p_u $$ 





### Let's get the whole matrix!

In [ ]:
P.dot(Q.T)

### Look at those results

Are they _exactly_ correct?
![check](img/check.gif)

## ALS benefit: Loss Function

The Loss function $L$ can be calculated as:

$$ L = \sum_{u,i ∈ \kappa}(r_{u,i}− q_i^T p_u)^2 + λ( ||q_i||^2 + |p_u||^2)$$

Where $\kappa$ is the set of (u,i) pairs for which $r_{u,i}$ is known.

To avoid overfitting, the loss function also includes a regularization parameter $\lambda$. We will choose a $\lambda$ to minimize the square of the difference between all ratings in our dataset $R$ and our predictions.

There's the **least squares** part of ALS, got it!

## So now we use gradient descent, right?

![incorrect](img/incorrect.gif)

### Here comes the alternating part

ALS alternates between holding the $q_i$'s constant and the $p_u$'s constant. 

While all $q_i$'s are held constant, each $p_u$ is computed by solving the least squared problem.<br>
After that process has taken place, all the $p_u$'s are held constant while the $q_i$'s are altered to solve the least squares problem, again, each independently.<br> 
This process repeats many times until you've reached convergence (ideally).

### Changing Loss function:

First let's assume first the item vectors are fixed, we first solve for the user vectors:

$$p_u=(\sum{r{u,i}\in r_{u*}}{q_iq_i^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{q_{i}}}$$__
Then we hold the user vectors constant and solve for the item vectors

$$q_i=(\sum{r{u,i}\in r_{i*}}{p_up_u^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{p_{u}}}$$__
This process repeats until convergence

# Review
What levers do we have available to adjust?
![lever-choice](img/levers.jpeg)

- Pros and cons of large rank?
- Pros and cons of lambda size?
- Iterations?

# Enough - let's get to the data

### Importing the Data
To begin with:
* initialize a SparkSession object
* import the dataset found at './data/ratings.csv' into a pyspark DataFrame

In [7]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [8]:
!ls data/

movies.csv  ratings.csv


In [9]:
!head data/ratings.csv

userId,movieId,rating,timestamp
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931
1,70,3.0,964982400
1,101,5.0,964980868
1,110,4.0,964982176
1,151,5.0,964984041


In [10]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/ratings.csv',
                               inferSchema=True,
                               header=True)

Check the data types of each of the values to ensure that they are a type that makes sense given the column.

In [11]:
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [14]:
movie_ratings.head(5)

[Row(userId=1, movieId=1, rating=4.0, timestamp=964982703),
 Row(userId=1, movieId=3, rating=4.0, timestamp=964981247),
 Row(userId=1, movieId=6, rating=4.0, timestamp=964982224),
 Row(userId=1, movieId=47, rating=5.0, timestamp=964983815),
 Row(userId=1, movieId=50, rating=5.0, timestamp=964982931)]

In [12]:
movie_ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



We aren't going to need the time stamp, so we can go ahead and remove that column.

In [15]:
movie_ratings = movie_ratings.drop('timestamp')

In [16]:
movie_ratings.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



### Fitting the Alternating Least Squares Model

Because this dataset is already preprocessed for us, we can go ahead and fit the Alternating Least Squares model.

* Use the randomSplit method on the pyspark DataFrame to separate the dataset into a training and test set
* Import the ALS module from pyspark.ml.recommendation.
* Fit the Alternating Least Squares Model to the training dataset. Make sure to set the userCol, itemCol, and ratingCol to the appropriate names given this dataset. Then fit the data to the training set and assign it to a variable model. 

In [19]:
# split into training and testing sets

mr_train, mr_test = movie_ratings.randomSplit([0.8, 0.2])

In [17]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

In [20]:
# Build the recommendation model using ALS on the training data
# fit the ALS model to the training set

als_model = als.fit(mr_train)

Now you've fit the model, and it's time to evaluate it to determine just how well it performed.

* import `RegressionEvalutor` from pyspark.ml.evaluation ([documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator)
* generate predictions with your model for the test set by using the `transform` method on your ALS model
* evaluate your model and print out the RMSE from your test set [options for evaluating regressors](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator.metricName)

In [21]:
train_predictions = als_model.transform(mr_train)

In [22]:
train_predictions.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   191|    148|   5.0|  4.927011|
|   133|    471|   4.0| 3.2046044|
|   597|    471|   2.0| 3.7945027|
|   385|    471|   4.0| 3.0570471|
|    91|    471|   1.0| 2.2039912|
+------+-------+------+----------+
only showing top 5 rows



In [24]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(train_predictions.dropna())
print(f"Root-mean-square error (train)= {rmse}")



Root-mean-square error (train)= 0.5644447976319404


In [23]:
predictions = als_model.transform(mr_test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions.dropna())
print(f"Root-mean-square error = {rmse}")



Root-mean-square error = 0.8821754924340985


In [25]:
predictions.persist()

DataFrame[userId: int, movieId: int, rating: double, prediction: float]

In [27]:
movie_ratings.show(6)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
+------+-------+------+
only showing top 6 rows



In [28]:
predictions.show(6)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   436|    471|   3.0| 3.8529267|
|   602|    471|   4.0| 2.9285228|
|   520|    471|   5.0|    3.3134|
|   448|    471|   4.0| 3.0964446|
|   312|    471|   4.0| 3.7356584|
|   216|    471|   3.0|  3.885703|
+------+-------+------+----------+
only showing top 6 rows



In [29]:
user_factors = als_model.userFactors

In [33]:
user_factors.show(10)

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[-0.028163323, 1....|
| 20|[0.5673277, 0.496...|
| 30|[0.58020854, 0.78...|
| 40|[0.7170096, 0.631...|
| 50|[0.2811989, 0.340...|
| 60|[0.5366256, 0.700...|
| 70|[0.542207, 1.0128...|
| 80|[0.7283111, 0.597...|
| 90|[1.2944326, 0.097...|
|100|[0.73494244, 0.66...|
+---+--------------------+
only showing top 10 rows



In [31]:
item_factors = als_model.itemFactors

In [34]:
item_factors.show(10)

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[0.3847322, 0.807...|
| 20|[0.70699704, 0.94...|
| 30|[0.6348481, 0.165...|
| 40|[0.4589276, -0.12...|
| 50|[0.8616392, 0.747...|
| 60|[0.5515125, 0.701...|
| 70|[1.2577468, -0.01...|
| 80|[0.6311168, 0.457...|
|100|[0.44572285, 0.93...|
|110|[0.74320126, 1.35...|
+---+--------------------+
only showing top 10 rows



### Important Question

Will Billy like movie m?

In [35]:
import numpy as np

In [36]:
billy_row = user_factors[user_factors['id'] == 10].first()
billy_factors = np.array(billy_row['features'])

In [47]:
m_row = item_factors[item_factors['id'] == 300].first()
m_factors = np.array(m_row['features'])

In [48]:
billy_factors

array([-0.02816332,  1.36987567, -1.05537736, -0.35999161,  0.53521347,
        0.6207878 ,  0.6986748 ,  0.48964936, -0.39119387,  0.31586847])

In [49]:
m_factors

array([ 0.65726769,  0.74716783, -0.10188741, -0.52062732,  1.09512174,
        0.83988464,  0.32136059,  0.02514426,  1.04226363,  0.09988138])

In [50]:
billy_factors @ m_factors

2.268142018317759

In [51]:
billy_factors.dot(m_factors)

2.268142018317759

In [44]:
billy_preds = train_predictions[train_predictions['userId'] == 10]

In [45]:
billy_preds.sort('movieId').show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    10|    296|   1.0|  2.018721|
|    10|    356|   3.5| 3.7405708|
|    10|    588|   4.0| 3.1175637|
|    10|    597|   3.5| 3.5979097|
|    10|   1028|   0.5| 2.7993033|
|    10|   1247|   3.0|  2.197189|
|    10|   1307|   3.0| 3.1363187|
|    10|   1784|   3.5|  3.451641|
|    10|   1907|   4.0|  3.064485|
|    10|   2571|   0.5|   2.64204|
|    10|   2762|   0.5| 2.2830927|
|    10|   2858|   1.0| 1.8638167|
|    10|   3882|   3.0| 2.8551207|
|    10|   4246|   3.5| 3.3944802|
|    10|   4306|   4.5| 3.8531947|
|    10|   4447|   4.5|  3.582965|
|    10|   4993|   4.0| 2.6037142|
|    10|   5066|   3.0| 2.8447907|
|    10|   5377|   3.5|  3.005398|
|    10|   5620|   3.0| 3.0089653|
+------+-------+------+----------+
only showing top 20 rows



In [52]:
!grep "^300," < data/movies.csv

300,Quiz Show (1994),Drama


## Okay, what *will* Billy like?

In [53]:
recs = als_model.recommendForAllUsers(numItems=10)

In [54]:
recs[recs['userId']==10].first()['recommendations']

[Row(movieId=179819, rating=4.995639324188232),
 Row(movieId=71579, rating=4.865113258361816),
 Row(movieId=5466, rating=4.772733211517334),
 Row(movieId=111617, rating=4.772619724273682),
 Row(movieId=140110, rating=4.7544732093811035),
 Row(movieId=112804, rating=4.722951412200928),
 Row(movieId=714, rating=4.664199352264404),
 Row(movieId=34332, rating=4.6507248878479),
 Row(movieId=82, rating=4.609297752380371),
 Row(movieId=85367, rating=4.600451469421387)]

In [56]:
!grep "^71579," < data/movies.csv

71579,"Education, An (2009)",Drama|Romance


## Objective Review

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* Run a brief hyper-parameter selection activity through a scalable grid search
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Some great technical resources:

- [good one from Stanford](http://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf)
- [the netflix recommendation project](https://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf)